In [2]:
words = open('names.txt', 'r').read().splitlines()

In [107]:
len(words)

32033

In [20]:
# bigram dictionary

b = {}

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs,chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) +1

len(b)


627

In [21]:
# trigram

t = {}

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        
        trigram = (ch1,ch2,ch3)
        t[trigram] = t.get(trigram,0) +1
        
len(t)

6037

In [109]:
len(t)

6037

In [110]:
sorted(t.items(),key=lambda kv: -kv[1])

[(('a', 'h', '.'), 1714),
 (('n', 'a', '.'), 1673),
 (('a', 'n', '.'), 1509),
 (('o', 'n', '.'), 1503),
 (('.', 'm', 'a'), 1453),
 (('.', 'j', 'a'), 1255),
 (('.', 'k', 'a'), 1254),
 (('e', 'n', '.'), 1217),
 (('l', 'y', 'n'), 976),
 (('y', 'n', '.'), 953),
 (('a', 'r', 'i'), 950),
 (('i', 'a', '.'), 903),
 (('i', 'e', '.'), 858),
 (('a', 'n', 'n'), 825),
 (('e', 'l', 'l'), 822),
 (('a', 'n', 'a'), 804),
 (('i', 'a', 'n'), 790),
 (('m', 'a', 'r'), 776),
 (('i', 'n', '.'), 766),
 (('e', 'l', '.'), 727),
 (('y', 'a', '.'), 716),
 (('a', 'n', 'i'), 703),
 (('.', 'd', 'a'), 700),
 (('l', 'a', '.'), 684),
 (('e', 'r', '.'), 683),
 (('i', 'y', 'a'), 669),
 (('l', 'a', 'n'), 647),
 (('.', 'b', 'r'), 646),
 (('n', 'n', 'a'), 633),
 (('.', 'a', 'l'), 632),
 (('.', 'c', 'a'), 628),
 (('r', 'a', '.'), 627),
 (('n', 'i', '.'), 625),
 (('.', 'a', 'n'), 623),
 (('n', 'n', '.'), 619),
 (('n', 'e', '.'), 607),
 (('e', 'e', '.'), 605),
 (('e', 'y', '.'), 602),
 (('.', 'k', 'e'), 601),
 (('a', 'l', 'e')

In [92]:
len(t)

6037

In [6]:
chars = sorted(set(''.join(words)))


In [7]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [3]:
import torch

tensor = torch.zeros((27,27,27),dtype=torch.int32)

In [14]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]

        tensor[ix1,ix2,ix3] += 1

In [15]:
tensor = tensor.float()
tensor += 0.1 # smoothing

tensor = tensor / tensor.sum(axis=2,keepdim=True)

In [84]:
g = torch.Generator().manual_seed(2147483647)
c = 0

for i in range(10):
    ix1 = 0
    ix2 = 0
    out = []
    temperature = 0.6

    while True:
        p = tensor[ix1,ix2] ** (1.0/temperature)
        ix3 = torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        if ix3 == 0:
            break
        out.append(itos[ix3])
        ix1, ix2 = ix2, ix3
        
    print(''.join(out))




juniden
pracar
pariah
olin
fair
lailia
sarie
umanna
uria
len


In [17]:
# calculating loss

log_likelihood = 0
n = 0

for w in words:
#for w in ['kiran']:

    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]

        prob = tensor[ix1,ix2,ix3]
        log_prob = torch.log(prob)
        log_likelihood += log_prob
        n += 1


print(f'{log_likelihood = }')
nll = -log_likelihood
print(f'{nll = }')
print(f'{nll/n}')


log_likelihood = tensor(-405251.6875)
nll = tensor(405251.6875)
2.0664193630218506


In [64]:
# Neural network approach
device = 'cuda' if torch.cuda.is_available() else 'cpu'


xs,ys = [],[]

for w in words:

    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]

        xs.append([ix1,ix2])
        ys.append(ix3)


xs = torch.tensor(xs, device=device)
ys = torch.tensor(ys, device=device)
num = xs.shape[0]




g = torch.Generator(device=device).manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True, device=device)



In [71]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'


xs = torch.tensor(xs, device=device)
ys = torch.tensor(ys, device=device)
num = xs.shape[0]

xenc = F.one_hot(xs, num_classes=27).float().to(device)  # shape: [num, 2, 27]
xenc = xenc.view(-1, 2 * 27)  # shape: [num, 54]

g = torch.Generator(device=device).manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True, device=device)


/tmp/ipykernel_79189/786169802.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xs = torch.tensor(xs, device=device)
/tmp/ipykernel_79189/786169802.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ys = torch.tensor(ys, device=device)


In [81]:
epoch = 0
for k in range(1000):
    # Forward pass
    logits = xenc @ W
    loss = F.cross_entropy(logits, ys)  
    print(f'{loss.item()=}, {epoch=}')
    # Backward pass
    W.grad = None
    loss.backward()


    W.data += -1.0 * W.grad  

   
    epoch += 1


loss.item()=2.2432398796081543, epoch=0
loss.item()=2.243238925933838, epoch=1
loss.item()=2.243238687515259, epoch=2
loss.item()=2.243237257003784, epoch=3
loss.item()=2.243237257003784, epoch=4
loss.item()=2.243236541748047, epoch=5
loss.item()=2.2432363033294678, epoch=6
loss.item()=2.2432353496551514, epoch=7
loss.item()=2.243234157562256, epoch=8
loss.item()=2.2432336807250977, epoch=9
loss.item()=2.2432334423065186, epoch=10
loss.item()=2.243232488632202, epoch=11
loss.item()=2.2432327270507812, epoch=12
loss.item()=2.243232250213623, epoch=13
loss.item()=2.243232011795044, epoch=14
loss.item()=2.243231773376465, epoch=15
loss.item()=2.2432310581207275, epoch=16
loss.item()=2.2432310581207275, epoch=17
loss.item()=2.2432303428649902, epoch=18
loss.item()=2.243229866027832, epoch=19
loss.item()=2.2432291507720947, epoch=20
loss.item()=2.2432284355163574, epoch=21
loss.item()=2.2432284355163574, epoch=22
loss.item()=2.24322772026062, epoch=23
loss.item()=2.2432267665863037, epoch=2

In [99]:
# predicting

g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    
    ix1 = 0
    ix2 = 0
    out = []
    temperature = 0.6

    while True:

        logits = xenc @ W
        count = logits.exp()
        probs = count / count.sum(1, keepdims=True)
        p = tensor[ix1,ix2] ** (1.0/temperature)

        ix3 = torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        if ix3 == 0:
            break
        out.append(itos[ix3])

        ix1, ix2 = ix2, ix3
        
    print(''.join(out))


juniden
pracar
pariah
olin
fair
lailia
sarie
umanna
uria
len
